In [7]:
import requests
from bs4 import BeautifulSoup
import time
from urllib import urlencode
import pandas as pd
from pymongo import MongoClient
from parse_clean_store import (parse_route_page, parse_user_page)

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client.routes_collection_updated
routes_collection = db.routes_collection
raw_data = routes_collection.find()
route_df = pd.DataFrame(list(raw_data))
route_df.head()

,_id,html,route,url
0,58d29f9540b4412f7b79585d,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",30(1) Feet of Pleasure,/v/301-feet-of-pleasure/106862592
1,58d29f9540b4412f7b79585f,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",41 Feet of Pain,/v/41-feet-of-pain/106862549
2,58d29f9540b4412f7b795861,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.5 Crack,/v/55-crack/108318876
3,58d29f9540b4412f7b795863,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 crack,/v/56-crack/106630563
4,58d29f9540b4412f7b795865,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 Dihedral,/v/56-dihedral/106165717


### parse and clean route info

In [11]:
dict_list = []
_id = 0
for html, url in zip(route_df['html'], route_df['url']):
    route_dict = parse_route_page(_id, html)
    route_dict['route_url'] = url # add url
    _id += 1 # update id
    dict_list.append(route_dict)

'''
df_routes = pd.DataFrame(dict_list)
df_routes.head(50).T
'''

'\ndf_routes = pd.DataFrame(dict_list)\ndf_routes.head(50).T\n'

### store route info

In [12]:
client = MongoClient('mongodb://localhost:27017/')
db = client.routes_updated
routes = db.routes
routes.insert_many(dict_list)

### clean and store user info

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client.users_info
users_info = db.users_info
raw_data = users_info.find()
df_user_html = pd.DataFrame(list(raw_data))
df_user_html.head()

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client.users
users = db.users
_id = 0
for html in df_user_html['html']:
    dict_list = parse_user_page(html, _id)
    users.insert_one(dict_list)
    _id += 1